In [1]:

import pandas as pd
from torch.utils.data import Dataset

from torch.utils.data import DataLoader
import torch
from torchvision import transforms
import torch.nn as nn
from torchvision import models
import torch
import pandas as pd
from PIL import Image

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [ ]:
class ImageEncoder(nn.Module):
    def __init__(self, embedding_size):
        super(ImageEncoder, self).__init__()
        self.resnet50 = models.resnet50(pretrained=True)
        self.fc = nn.Linear(self.resnet50.fc.out_features, embedding_size)
        
    def forward(self, x):
        x = self.resnet50(x)
        x = self.fc(x)
        return x

In [ ]:
model_name =  'model_image_2023-06-28.pth'
image_model = ImageEncoder(768).to(device)
image_model.load_state_dict(torch.load(model_name))
# image_model.eval()

<All keys matched successfully>

In [ ]:
class FDataset(Dataset):
    def __init__(self, annotations_file, start, end):
        self.data = pd.read_csv(annotations_file)[start:end]   
        self.transform =  transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        print(img_path)
        image = Image.open(img_path)
        if image.mode != 'RGB':
            image = image.convert('L')
            image = Image.merge('RGB', [image] * 3)
        image = self.transform(image)
        caption = self.data.iloc[idx, 1]
        return image, caption

In [6]:
start = 0
end = 50000
cnt = end - start
dataset = FDataset('./data/anotation_new.csv', start, end)
dataloader =  DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True, num_workers=5)
image_embeddings = torch.Tensor([])


In [7]:
next(image_model.parameters()).is_cuda

True

In [8]:
for i, (img, _) in enumerate(dataloader):
    # 予測と損失の計算
    print(i)
    img = img.to(device)
    # with torch.no_grad():
    image_embedding = image_model(img).cpu()
    image_embeddings = torch.cat((image_embeddings, image_embedding), dim=0)
    if i % 30 == 0:
        print(f"finish: {len(image_embeddings) * 100/len(dataloader)}%")

torch.save(image_embeddings, f'image_tensor/{model_name}/resnet50_tensor_{start}-{end}.pt')